In [1]:
from google.colab import auth
auth.authenticate_user()

from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
import os
path = 'gdrive/My Drive/Colab_Notebooks/cs224n-2019/a2'
os.chdir(path)

In [3]:
!dir

collect_submission.sh  get_datasets.sh	sgd.py	w2v-Skipgram.ipynb
env.yml		       run.ipynb	utils	word2vec.py


In [4]:
!sudo bash get_datasets.sh

--2019-05-28 04:18:52--  http://nlp.stanford.edu/~socherr/stanfordSentimentTreebank.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/~socherr/stanfordSentimentTreebank.zip [following]
--2019-05-28 04:18:52--  https://nlp.stanford.edu/~socherr/stanfordSentimentTreebank.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6372817 (6.1M) [application/zip]
Saving to: ‘stanfordSentimentTreebank.zip’

stanfordSentimentTr 100%[===================>]   6.08M  3.41MB/s    in 1.8s    

2019-05-28 04:18:54 (3.41 MB/s) - ‘stanfordSentimentTreebank.zip’ saved [6372817/6372817]

Archive:  stanfordSentimentTreebank.zip
   creating: stanfordSentimentTreebank/
  inflating: stanfordSentimentTreebank/datasetSentences.txt  
   creating: 

In [0]:
import random
import numpy as np
import matplotlib
matplotlib.use('agg')
import matplotlib.pyplot as plt
import time

from utils.treebank import StanfordSentiment
from word2vec import *
from sgd import *

In [0]:
# Check Python Version
import sys
assert sys.version_info[0] == 3
assert sys.version_info[1] >= 5

In [0]:
# Reset the random seed to make sure that everyone gets the same results
random.seed(314)
dataset = StanfordSentiment()
tokens = dataset.tokens()
nWords = len(tokens)


In [8]:

# We are going to train 10-dimensional vectors for this assignment
dimVectors = 10

# Context size
C = 5

# Reset the random seed to make sure that everyone gets the same results
random.seed(31415)
np.random.seed(9265)

startTime=time.time()
wordVectors = np.concatenate(
    ((np.random.rand(nWords, dimVectors) - 0.5) /
       dimVectors, np.zeros((nWords, dimVectors))),
    axis=0)

wordVectors = sgd(
    lambda vec: word2vec_sgd_wrapper(skipgram, tokens, vec, dataset, C,
        negSamplingLossAndGradient),
    wordVectors, 0.3, 40000, None, True, PRINT_EVERY=1000)
# Note that normalization is not called here. This is not a bug,
# normalizing during training loses the notion of length.

print("sanity check: cost at convergence should be around or below 10")
print("training took %d seconds" % (time.time() - startTime))

# concatenate the input and output word vectors
wordVectors = np.concatenate(
    (wordVectors[:nWords,:], wordVectors[nWords:,:]),
    axis=0)

visualizeWords = [
    "great", "cool", "brilliant", "wonderful", "well", "amazing",
    "worth", "sweet", "enjoyable", "boring", "bad", "dumb",
    "annoying", "female", "male", "queen", "king", "man", "woman", "rain", "snow",
    "hail", "coffee", "tea"]

visualizeIdx = [tokens[word] for word in visualizeWords]
visualizeVecs = wordVectors[visualizeIdx, :]
temp = (visualizeVecs - np.mean(visualizeVecs, axis=0))
covariance = 1.0 / len(visualizeIdx) * temp.T.dot(temp)
U,S,V = np.linalg.svd(covariance)
coord = temp.dot(U[:,0:2])

for i in range(len(visualizeWords)):
    plt.text(coord[i,0], coord[i,1], visualizeWords[i],
        bbox=dict(facecolor='green', alpha=0.1))

plt.xlim((np.min(coord[:,0]), np.max(coord[:,0])))
plt.ylim((np.min(coord[:,1]), np.max(coord[:,1])))

plt.savefig('word_vectors.png')

iter 1000: 25.462040
iter 2000: 25.298383
iter 3000: 25.066112
iter 4000: 24.643775
iter 5000: 24.221150
iter 6000: 23.827730
iter 7000: 23.325909
iter 8000: 22.876001
iter 9000: 22.430623
iter 10000: 21.875617
iter 11000: 21.342645
iter 12000: 20.930986
iter 13000: 20.469180
iter 14000: 19.993677
iter 15000: 19.524647
iter 16000: 19.034326
iter 17000: 18.574380
iter 18000: 18.166311
iter 19000: 17.784676
iter 20000: 17.415276
iter 21000: 17.064078
iter 22000: 16.737958
iter 23000: 16.381199
iter 24000: 16.014223
iter 25000: 15.750859
iter 26000: 15.514099
iter 27000: 15.201966
iter 28000: 14.954310
iter 29000: 14.656397
iter 30000: 14.334909
iter 31000: 14.089950
iter 32000: 13.934838
iter 33000: 13.678010
iter 34000: 13.440420
iter 35000: 13.309861
iter 36000: 13.131846
iter 37000: 12.950496
iter 38000: 12.813099
iter 39000: 12.661149
iter 40000: 12.478976
sanity check: cost at convergence should be around or below 10
training took 7562 seconds
